In [8]:
import numpy as np
import reconstruction.utils.plot
import importlib; importlib.reload(reconstruction.utils.plot)
from reconstruction.utils.plot import plot_vol
from reconstruction.utils.data_prep import get_fractured

dataset = np.load('datasets/voxels.npy', allow_pickle=True).item()['train']

In [9]:
dataset['data'] = dataset['data'][:100]
dataset['labels'] = dataset['labels'][:100]

In [10]:
# random object
d = dataset['data'][np.random.choice(len(dataset['data']))]
fractured = get_fractured(d)

print('Fractured:')
plot_vol(fractured)
print('Source:')
plot_vol(d)

Fractured:


c:\Users\aldav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:489: DeprecationWarning:

plotly.graph_objs.Scene is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Scene




Source:


In [11]:
import multiprocessing, time
from concurrent.futures import ThreadPoolExecutor

n_cpu = multiprocessing.cpu_count()

In [12]:
def map_get_fractured(data, lbl):
	repetitions = 4
	return [get_fractured(data) for i in range(repetitions)], [lbl]*repetitions

In [14]:
dataset['data'].shape, dataset['labels'].shape

((100, 32, 32, 32), (100,))

In [18]:
t0 = time.time()
with ThreadPoolExecutor(max_workers=n_cpu) as executor:
    res = executor.map(map_get_fractured, dataset['data'], dataset['labels'])

voxels, labels = zip(*res)
print('%.2fs' % (time.time() - t0))
len(voxels), len(labels)

0.34s


(100, 100)

In [21]:
voxels = np.concatenate(voxels)
labels = np.concatenate(labels)

In [22]:
voxels.shape, labels.shape

((400, 32, 32, 32), (400,))

In [ ]:
output = {'train':{}, 'test':{}, 'errors': []}

output['train']['labels'] = np.concatenate([dataset['labels'], labels[::2]])
output['train']['data'] = np.concatenate([dataset['data'], voxels[::2]])

output['test']['labels'] = labels[1::2]
output['test']['data'] = voxels[1::2]

np.save('datasets/chairnet.npy', output)